## Symbology Conversion Example
This is a simple notebook I created for a customer, demonstrating how to convert ISINs (or other Symbology) to RICs using the Refinitiv Data Platform Symbology API.
Whilst, I will be working with ISINs and RICs, the API supports other types such as:
- RIC
- ISIN
- CUSIP
- SEDOL
- LEI (Legal Entity Identifier)
- PermID
- others...

Note that whilst the Symbology API is quiet flexible, it may not always be possible to convert a symbol from one type to another - based on the to/from type combination and asset class involved etc.

Please refer to the following, for further details:
- [Symbology API Reference](https://apidocs.refinitiv.com/Apps/ApiDocs#/details/L2Rpc2NvdmVyeS9zeW1ib2xvZ3kvdjE=/L2xvb2t1cA==/POST/README) - in particular, the section *'identifier types covered by asset class'*
- [Symbology API User Guide](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis/documentation#symbology-user-guide)

I will be using the [RDP Python Library](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-libraries) to keep things simple. The RDP Library is a ease of wrapper for the RDP APIs and is available in Python, .NET and soon in Typescript.

However, if you are not using one of the above languages, you can still access the Symbology API and the other RDP APIs via a HTTP RESTful interface - as [detailed here](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis/tutorials#introduction-to-the-request-response-api)

**NOTES:** 
- The Symbology API is only available bundled with other RDP Products - please speak to your  Refinitiv Account team for further Details
- Whilst the RDP Library has a built in *convert_symbols()* function, this call actually uses the RDP Search API - which requires a different licence.



In [ ]:
import refinitiv.dataplatform as rdp
import json
import os, sys

#### I am going to read my Access Credentials from my ENV variables - replace with your Access credentials...
For details on Access Credentials, please see the [RDP Library Quick Start](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-libraries/quick-start)


In [ ]:
APP_KEY = os.environ['APP_KEY'] # Required for RDP Platform and Desktop Session
RDP_LOGIN = os.environ['RDP_LOGIN'] # Required for RDP Platform Session
RDP_PASSWORD= os.environ['RDP_PASSWORD'] # Required for RDP Platform Session

### Create a session to connect to Refinitiv Data Platform

In [ ]:
session = rdp.open_platform_session(APP_KEY,
    rdp.GrantPassword( username=RDP_LOGIN, 
                      password=RDP_PASSWORD )
)

### OR - for Eikon/Workspace application running on the same PC

In [ ]:
session = rdp.open_desktop_session(APP_KEY)

In [ ]:
# Read my ISINs in from a file - replace with your input symbols...
try:
    with open('SampleISINs.txt', 'r') as f:  # Read one RIC per line from file
            isins = [isin.strip(' \t\n\r') for isin in f]  # and strip any whitespace etc
except FileNotFoundError as fnf_error:
        print(fnf_error)

#### 1st pass - try and get the Primary RIC for each ISIN. 
#### Should cover most ISINs 

In [ ]:
symbologylookup_endpoint = rdp.Endpoint(session, 
    'https://api.refinitiv.com/discovery/symbology/v1/lookup')

request_body={
  "from": [
    {
      "identifierTypes": [
        "ISIN"
      ],
      "values": isins
    }
  ],
  "to": [
    {
      "identifierTypes": [
        "RIC"
      ]
    }
  ],
  "path": [
    {
      "relationshipTypes": [
        "InverseIsValuationQuoteOf"
      ],
      "objectTypes": [
        {
          "from": "AnyInstrument",
          "to": "AnyQuote"
        }
      ]
    }
  ],
  "type": "strict"
}


response = symbologylookup_endpoint.send_request(
    method = rdp.Endpoint.RequestMethod.POST,
    body_parameters = request_body
)

print(json.dumps(response.data.raw, indent=2))


If you get a reponse which says something like:   
    <p>*'code': 'insufficient_scope', 'message': 'access denied.*   </p>
this means your are not licenced to use the Symbology API - Please speak to your Refinitiv Account team

In [ ]:
# Extract the data from the response
mydata = response.data.raw['data']

In [ ]:
# Check results to see if conversion to Primary RIC failed for any ISINs
no_primary =[]
print('ISIN\t\tRIC')
for symbol in mydata:
    if symbol['output']:
        print(symbol['input'][0]['value'], symbol['output'][0]['value'])
    else:
        print(symbol['input'][0]['value'], "No Match")
        no_primary.append(symbol['input'][0]['value'])


In [ ]:
# Output list of ISINs with no Primary RIC
display(no_primary)

#### Try and match with non-Primary RICs
For many ISINs, we can expect to get back multiple RICs per ISIN  i.e. not a 1 to 1 relationship

In [ ]:
request_body={
  "from": [
    {
      "identifierTypes": [
        "ISIN"
      ],
      "values": no_primary
    }
  ],
  "to": [
    {
      "identifierTypes": [
        "RIC"
      ]
    }
  ],
  "type": "auto"
}

response = symbologylookup_endpoint.send_request(
    method = rdp.Endpoint.RequestMethod.POST,
    body_parameters = request_body
)

print(json.dumps(response.data.raw, indent=2))

In [ ]:
mydata = response.data.raw['data']

#### Output the first matching RIC for each ISIN
NOTE: The first matching RIC may not always be the one you require - you can contact the [My.Refinitiv Content Helpdesk](https://my.refinitiv.com/content/mytr/en/productsupport.html) for assistance with selecting the most appropriate RIC.

In [ ]:

print('ISIN\t\tRIC')
for symbol in mydata:
    if len(symbol['output'])==0:
        print(symbol['input'][0]['value'], "No Match")
    else:
        print(symbol['input'][0]['value'], symbol['output'][0]['value'])
        

In [ ]:
session.close()